In [1]:
# Cryptanalysis - Task 2 - CipherD and Attack - v1

# TO-DO:
#  - deduce the single bit in the k4 round from the counters

import numpy as np

#febc6d78039a4215
# different than in CipherThree and CipherFour !!!
def sbox(input):
    list_out = [15, 14, 11, 12, 6, 13, 7, 8, 0, 3, 9, 10, 4, 2, 1, 5]
    return list_out[input]

def inverse_sbox(input):
    list_out = [15, 14, 11, 12, 6, 13, 7, 8, 0, 3, 9, 10, 4, 2, 1, 5]
    return list_out.index(input)

def sbox_layer(input):
    out = []
    for i in range(4):
        #print((input >> (i * 4) % 16))
        out.append(sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def sbox_layer_inverse(input):
    out = []
    for i in range(4):
        #print(((input >> (i * 4)) % 16))
        out.append(inverse_sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def permutation(inputs):
    inp = inputs
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm[15 - i]] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)

def inverse_permutation(input):
    inp = input
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm.index(15 - i)] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)


def encrypt(keys, message):
    ct = message
    for i in range(len(keys) - 2):
        ct ^^= keys[i]
        ct = sbox_layer(ct)
        ct = permutation(ct)
    ct ^^= keys[-2]
    ct = sbox_layer(ct)
    ct ^^= keys[-1]
    return ct

def decrypt(keys, ct):
    pt = ct
    pt ^^= keys[-1]
    pt = sbox_layer_inverse(pt)
    pt ^^= keys[-2]
    for i in range(len(keys) - 3, -1, -1):
        pt = inverse_permutation(pt)
        pt = sbox_layer_inverse(pt)
        pt ^^= keys[i]
    return pt

def generate_pts_cts(keys, amount):
    cts = []
    for i in range(amount):
        ct = encrypt(keys, i)
        cts.append((i, ct))
    return cts

def generate_pts_cts_v2(keys, amount):
    cts = []
    for i in range(0, amount * 16**3, 16**3):
        ct = encrypt(keys, i)
        cts.append((i, ct))
    return cts

def attack(keys, mask):
    
    mask = 8 * 16**3  # ~[8, 0, 0, 0]
    pts_cts = generate_pts_cts(keys, 16**4)  # max 16**4
    #pts_cts = generate_pts_cts_v2(keys, 16)  # max 16**4
        
    guesses = range(0, 16**4, 16**3)
    #table = [[0, 0]] * len(guesses)
    table = {}
    
    for key_guess in guesses:
        table[key_guess] = [0, 0]
        for (pt, ct) in pts_cts:
            y_guess = sbox_layer_inverse(ct ^^ key_guess)
            
            a_0, a_1 = (pt & mask), (y_guess & mask)
            res = 0
            for i in range(16):
                res ^^= ((a_0 >> i) % 2) ^^ ((a_1 >> i) % 2)  # sum bits of a_0 and a_1 (mod 2)
            if res == 0:
                table[key_guess][0] += 1
            else:
                table[key_guess][1] += 1
    
    max_val = 0
    final_guesses = []
    for key in table.keys():
        values = table[key]
        #print(key, values)
        val = abs(values[0] - values[1])
        if val == max_val:
            final_guesses.append(key)
        elif val > max_val:
            final_guesses = [key]
            max_val = val
    
    #print("final guesses:", final_guesses)
    
    return table, final_guesses


mes_1 = 6
keys_1 = [1, 2, 3, 4, 7*16**3]
#keys_1 = [54241, 65414, 22836, 39868, 29301, 32817]
enc_1 = encrypt(keys_1, mes_1)
dec_1 = decrypt(keys_1, enc_1)

print("Enc and Dec:", bin(mes_1), "->", bin(enc_1), "->", bin(dec_1))

table_1, final_1 = attack(keys_1, 0)
print(table_1)
print("Guess(es):", final_1, "Correct:", keys_1[-1])

Enc and Dec: 0b110 -> 0b1110100101011001 -> 0b110
{0: [31032, 34504], 4096: [33184, 32352], 8192: [32392, 33144], 12288: [30240, 35296], 16384: [29544, 35992], 20480: [32704, 32832], 24576: [30040, 35496], 28672: [26880, 38656], 32768: [35968, 29568], 36864: [35944, 29592], 40960: [35520, 30016], 45056: [35544, 29992], 49152: [32480, 33056], 53248: [37816, 27720], 57344: [35168, 30368], 61440: [29832, 35704]}
Guess(es): [28672] Correct: 28672
